In [6]:
import json
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import requests


In [108]:
#spotify_token = "BQDNJ2LaBG2ABjf7KWJrvqHe-x_BdTWITzp5xH8ErwX-ne3fS4J3fjb2dz1qSBpihMp1AOCMRA5J6xbuHdGQcX04WWE-iSRljUQdKsXFasqPcHN_qxx541121hreRXpw15hvUZKoZxaQQ9tcWUA6l2pZOworr6gGcOExwz_auhCH9fw"
spotify_user_id = "1111362677"
spotify_token = "BQClB_6ybwNanbCObt1WdUskvCMXfQ7Q3ogwxh1cLGVcGNbfRVOJOXfpQBM1LxwdBK6RprPhi247gawXCmuOfaEijRwHy9r3kQLdjRckHQVPYtLYkVtNa0ZZmRvHJshSL5kc0dnjwVcyPSOLyMqicncQkmX2oKRKgL0i-It9oMEMgYud8R7ixMB8BxkGxfOLBQIWl7sFOcT05PMVL3e1nxxDq8ad"

In [8]:
"""Create A New Playlist"""
request_body = json.dumps({
    "name": "Youtube Liked Vids",
    "description": "All Liked Youtube Videos",
    "public": True
    })

print(request_body)

query = "https://api.spotify.com/v1/users/{}/playlists".format(spotify_user_id)

print(query)

response = requests.post(
    query,
    data=request_body,
    headers={
    "Content-Type": "application/json",
    "Authorization": "Bearer {}".format(spotify_token)
    }
    )
print(response)

#response_json = response.json()

{"name": "Youtube Liked Vids", "description": "All Liked Youtube Videos", "public": true}
https://api.spotify.com/v1/users/1111362677/playlists
<Response [403]>


In [15]:
 """Create A New Playlist"""
request_body = json.dumps({
    "name": "SurfSound_V1",
    "description": "Jam to the sounds of your surfing history!",
    "public": False
    })

query = "https://api.spotify.com/v1/users/{}/playlists".format(spotify_user_id)
response = requests.post(
    query,
    data=request_body,
    headers={
    "Content-Type": "application/json",
    "Authorization": "Bearer {}".format(spotify_token)
    }
    )
response_json = response.json()

print(request_body)
print(query)


{"name": "SurfSound_V1", "description": "Jam to the sounds of your surfing history!", "public": false}
https://api.spotify.com/v1/users/1111362677/playlists


In [41]:
query = "https://api.spotify.com/v1/users/{}/playlists".format(spotify_user_id)

response = requests.get(
        query,
        data=request_body,
        headers={
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(spotify_token)
        })
#response_json = response.json()
print(response)

<Response [400]>


In [139]:
class CreatePlaylist:
    def __init__(self):
        # Get your starting info (Spotify)
        self.playlist_name = 'SurfSound'
        self.all_song_info = {}

    def create_playlist(self):
        """Create A New Playlist"""
        request_body = json.dumps({
        "name": self.playlist_name,
        "description": "Jam to the sounds of your surfing history!",
        "public": True
        })

        query = "https://api.spotify.com/v1/users/{}/playlists".format(spotify_user_id)
        response = requests.post(
        query,
        data=request_body,
        headers={
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(spotify_token)
        }
        )
        response_json = response.json()

        # playlist id
        return response_json["id"]
    
    def get_spotify_uri(self, song_name, artist):
        """Search For the Song"""
        query = "https://api.spotify.com/v1/search?query=track%3A{}+artist%3A{}&type=track&offset=0&limit=20".format(
        song_name,
        artist
        )
        response = requests.get(
        query,
        headers={
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(spotify_token)
        }
        )
        
        response_json = response.json()
        songs = response_json["tracks"]["items"]
        
        # only use the first song
        uri = songs[0]["uri"]

        return uri
    
    def check_playlist(self):
        response = requests.get(
            'https://api.spotify.com/v1/users/{}/playlists'.format(spotify_user_id),
            headers={
            "Content-Type": "application/json",
            "Authorization": "Bearer {}".format(spotify_token)
            }
            )
        
        allplaylists = [info for playlist, info in response.json().items()]
        idx = [names['id'] for names in allplaylists[1] if names['name'] == self.playlist_name]
        
        if not idx:
            playlist_exists = 0
            playlist_id = 0
        else:
            playlist_exists = 1
            playlist_id = idx[0]
        
        return playlist_exists, playlist_id

    def add_song_to_playlist(self):
        """Add all liked songs into a new Spotify playlist"""
        # populate dictionary with our liked songs
        #self.get_liked_videos()
        
        self.all_song_info['Song1'] = {
            "youtube_url": 'https://www.youtube.com/watch?v=IRDMInkkw7U',
            "song_name": 'Bitches',
            "artist": 'Tove Lo',
            "spotify_uri": self.get_spotify_uri('Bitches', 'Tove Lo')
            }
        
        self.all_song_info['Song2'] = {
            "youtube_url": 'https://www.youtube.com/watch?v=_Yhyp-_hX2s',
            "song_name": 'Lose Yourself',
            "artist": 'Eminem',
            "spotify_uri": self.get_spotify_uri('Lose Yourself', 'Eminem')
            }
        
        
        # collect all of uri
        uris = [info["spotify_uri"]
            for song, info in self.all_song_info.items()]
        
        # create a new playlist
        playlist_exists, playlist_id = self.check_playlist()
        
        if not playlist_exists:
            playlist_id = self.create_playlist()
        
        # add all songs into new playlist
        request_data = json.dumps(uris)
        query = "https://api.spotify.com/v1/playlists/{}/tracks".format(
            playlist_id)
        
        response = requests.post(
            query,
            data=request_data,
            headers={
            "Content-Type": "application/json",
            "Authorization": "Bearer {}".format(spotify_token)
            }
        )
        
        # check for valid response status
        if response.status_code != 200:
            raise Warning(response.status_code)

        response_json = response.json()
        return response_json

print(__name__)
if __name__ == '__main__':
    cp = CreatePlaylist()
    cp.add_song_to_playlist()


__main__


Warning: 201

In [118]:
request_body = json.dumps({
    "name": "SurfSound",
    })

response = requests.get(
    'https://api.spotify.com/v1/users/{}/playlists'.format(spotify_user_id),
    headers={
    "Content-Type": "application/json",
    "Authorization": "Bearer {}".format(spotify_token)
    }
    )

allplaylists = [info for playlist, info in response.json().items()]
allnames = [names['name'] for names in allplaylists[1]]

In [119]:
allnames

['SurfSound',
 'Winterflug 2020',
 'OUT & LOUD!',
 'Fietsen',
 'Nostalgia',
 'Favorieten',
 'Relax',
 'La piccola chiocciola',
 'Happy']

In [100]:
[names[id] for names in allplaylists[1]]

[{'collaborative': False,
  'description': '',
  'external_urls': {'spotify': 'https://open.spotify.com/playlist/2HxrYn24ZHyCtSFNDLVpk5'},
  'href': 'https://api.spotify.com/v1/playlists/2HxrYn24ZHyCtSFNDLVpk5',
  'id': '2HxrYn24ZHyCtSFNDLVpk5',
  'images': [{'height': 640,
    'url': 'https://mosaic.scdn.co/640/ab67616d0000b273189cbfd9b6d4a49901cfef3bab67616d0000b2732a73b3592817536ffa7217c9ab67616d0000b2736e6f8bf26d51894c12612e3eab67616d0000b273e268a664bdd12926971eeceb',
    'width': 640},
   {'height': 300,
    'url': 'https://mosaic.scdn.co/300/ab67616d0000b273189cbfd9b6d4a49901cfef3bab67616d0000b2732a73b3592817536ffa7217c9ab67616d0000b2736e6f8bf26d51894c12612e3eab67616d0000b273e268a664bdd12926971eeceb',
    'width': 300},
   {'height': 60,
    'url': 'https://mosaic.scdn.co/60/ab67616d0000b273189cbfd9b6d4a49901cfef3bab67616d0000b2732a73b3592817536ffa7217c9ab67616d0000b2736e6f8bf26d51894c12612e3eab67616d0000b273e268a664bdd12926971eeceb',
    'width': 60}],
  'name': 'Winterflug 2020

In [130]:
allplaylists = [info for playlist, info in response.json().items()]
allnames = [names['id'] for names in allplaylists[1] if names['name'] == 'SurfSoun']
allnames

[]

In [134]:
not allnames

True